In [2]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

Current directory: /home/ubuntu/arga-arc/tf_coder
Root directory: /home/ubuntu/arga-arc


In [76]:
import typing as t
import json
from pprint import pprint
from dataclasses import dataclass
import tensorflow as tf
import numpy as np
import math
from config import CONFIG
from openai import OpenAI
import re
from collections import Counter

pprint(CONFIG.__dict__.keys())

OPENAI = OpenAI(api_key=CONFIG.OPENAI_SECRET_KEY, organization=CONFIG.OPENAI_ORGANIZATION)

dict_keys(['OPENAI_SECRET_KEY', 'OPENAI_ORGANIZATION'])


## parsing the dataset

In [19]:
class OutputJSON(t.TypedDict):
    task_id: str
    completions: t.List[str]
    coverage_percentage: float
    description: str
    tf_operators: t.Dict[str, int]
    total_covered: int
    total_in_target: int

class ExamplesJSON(t.TypedDict):
    inputs: str
    outputs: str

class TaskJSON(t.TypedDict):
    constants: str
    description: str
    name: str
    source: str
    target_program: str
    examples: ExamplesJSON

DATASET_FILE = CURRENT_DIRECTORY / "tfcoder_dataset.json"
DATASET: t.List[TaskJSON] = json.loads(DATASET_FILE.read_text())

print(f"Loaded {len(DATASET)} tasks from {DATASET_FILE}")

TASK_JSONS = {task["name"] : task for task in DATASET}
pprint(TASK_JSONS.keys())

Loaded 72 tasks from /home/ubuntu/arga-arc/tf_coder/tfcoder_dataset.json
dict_keys(['google_01', 'google_02', 'google_03', 'google_04', 'google_05', 'google_06', 'google_07', 'google_08', 'google_09', 'google_10', 'google_11', 'google_12', 'google_13', 'google_14', 'google_15', 'google_16', 'google_17', 'google_18', 'google_19', 'google_20', 'google_21', 'google_22', 'stackoverflow_01', 'stackoverflow_02', 'stackoverflow_03', 'stackoverflow_04', 'stackoverflow_05', 'stackoverflow_06', 'stackoverflow_07', 'stackoverflow_08', 'stackoverflow_09', 'stackoverflow_10', 'stackoverflow_11', 'stackoverflow_12', 'stackoverflow_13', 'stackoverflow_14', 'stackoverflow_15', 'stackoverflow_16', 'stackoverflow_17', 'stackoverflow_18', 'stackoverflow_19', 'stackoverflow_20', 'stackoverflow_21', 'stackoverflow_22', 'stackoverflow_23', 'stackoverflow_24', 'stackoverflow_25', 'stackoverflow_26', 'stackoverflow_27', 'stackoverflow_28', 'stackoverflow_29', 'stackoverflow_30', 'stackoverflow_31', 'stackover

In [48]:
@dataclass
class Example:
    inputs: t.List[np.ndarray]
    output: t.Union[np.ndarray, tf.SparseTensor]

    @classmethod
    def from_json(cls, examples: ExamplesJSON):
        try:
            evaluated_inputs = eval(examples["inputs"])
            if isinstance(evaluated_inputs, list):
                inputs = [np.array(i) for i in evaluated_inputs]
            else:
                inputs = [evaluated_inputs]
        except Exception as e:
            print(f"Error evaluating inputs: {e}")
            print(f"Inputs: {examples['inputs']}")
            raise e

        try:
            evaluated_outputs = eval(examples["outputs"])
            if isinstance(evaluated_outputs, list):
                outputs = np.array(evaluated_outputs)
            elif isinstance(evaluated_outputs, tf.SparseTensor):
                outputs = evaluated_outputs
            elif isinstance(evaluated_outputs, tf.Tensor):
                outputs = evaluated_outputs.numpy()
            else:
                outputs = evaluated_outputs
        except Exception as e:
            print(f"Error evaluating outputs: {e}")
            print(f"Outputs: {examples['outputs']}")
            raise e

        return cls(inputs, outputs)
    
    def toJSON(self):
        return {
            "inputs": [i.tolist() for i in self.inputs],
            "output": self.output.tolist()
        }

for name, task in TASK_JSONS.items():
    task["parsed_examples"] = Example.from_json(task["examples"])

pprint(TASK_JSONS["google_03"])


{'completions': ['tf.sparse.slice(input_1, start=[0,0,0], size=[1,-1,-1])',
                 'tf.sparse.slice(input_1, [0,0,0], [1,-1,-1])',
                 'tf.sparse.slice(input_1, tf.constant([0, 0, 0], '
                 'dtype=tf.int64), tf.constant([1, -1, -1], dtype=tf.int64))',
                 'tf.sparse.slice(input_1, [0,0,0], [1,-1,-1])',
                 'tf.sparse.slice(input_1, [0, 0, 0], [1, -1, -1])',
                 'tf.sparse.slice(input_1, [0, 0, 0], [1, -1, -1])',
                 'tf.sparse.slice(input_1, [0, 0, 0], [1, -1, -1])',
                 'tf.sparse.slice(input_1, [0, 0, 0], [1, -1, -1])',
                 'tf.sparse.slice(input_1, [0, 0, 0], [1, -1, -1])',
                 'tf.sparse.slice(input_1, [0, 0, 0], [1, -1, -1])'],
 'constants': '[]',
 'description': 'Slice the first dimension of a SparseTensor',
 'examples': {'inputs': 'tf.SparseTensor(indices=[[0, 0, 0], [0, 1, 1], [1, 1, '
                        '1], [1, 1, 2]], values=[1., 1., 1., 1.], '


## prompt

In [38]:
TFOPERATORS = "\nTensorFlow functions to use:\n---------------------\ntf.abs(x)\ntf.add(x, y)\ntf.add_n(inputs)\ntf.argmax(input, axis)\ntf.argmin(input, axis)\n"+\
"tf.argsort(values, axis, stable=True)\ntf.argsort(values, axis, direction='DESCENDING', stable=True)\ntf.boolean_mask(tensor, mask)\ntf.broadcast_to(input, shape)\n"+\
"tf.cast(x, dtype)\ntf.clip_by_value(t, clip_value_min, clip_value_max)\ntf.concat(values, axis)\ntf.constant(value)\ntf.constant(value, dtype)\ntf.divide(x, y)\n"+\
"tf.equal(x, y)\ntf.exp(x)\ntf.expand_dims(input, axis)\ntf.eye(num_rows)\ntf.eye(num_rows, num_columns)\ntf.eye(num_rows, dtype)\ntf.fill(dims, value)"+\
"tf.gather(params, indices)\ntf.gather(params, indices, axis, batch_dims)\ntf.gather_nd(params, indices)\ntf.gather_nd(params, indices, batch_dims)\ntf.greater(x, y)\n"+\
"tf.greater_equal(x, y)\ntf.math.bincount(arr)\ntf.math.ceil(x)\ntf.math.count_nonzero(input)\ntf.math.count_nonzero(input, axis)\ntf.math.cumsum(x, axis)\n"+\
"tf.math.cumsum(x, axis, exclusive=True)\ntf.math.divide_no_nan(x, y)\ntf.math.floor(x)\ntf.math.log(x)\ntf.math.logical_and(x, y)\ntf.math.logical_not(x)"+\
"tf.math.logical_or(x, y)\ntf.math.logical_xor(x, y)\ntf.math.negative(x)\ntf.math.reciprocal(x)\ntf.math.reciprocal_no_nan(x)\ntf.math.segment_max(data, segment_ids)\n"+\
"tf.math.segment_mean(data, segment_ids)\ntf.math.segment_min(data, segment_ids)\ntf.math.segment_prod(data, segment_ids)\ntf.math.segment_sum(data, segment_ids)\n"+\
"tf.math.squared_difference(x, y)\ntf.math.top_k(input, k)\ntf.math.unsorted_segment_max(data, segment_ids, num_segments)\ntf.math.unsorted_segment_mean(data, segment_ids, num_segments)\n"+\
"tf.math.unsorted_segment_min(data, segment_ids, num_segments)\ntf.math.unsorted_segment_prod(data, segment_ids, num_segments)\ntf.math.unsorted_segment_sum(data, segment_ids, num_segments)\n"+\
"tf.matmul(a, b)\ntf.maximum(x, y)\ntf.minimum(x, y)\ntf.multiply(x, y)\ntf.not_equal(x, y)\ntf.one_hot(indices, depth)\ntf.ones(shape)\ntf.ones_like(input)\n"+\
"tf.pad(tensor, paddings, mode='CONSTANT')\ntf.pad(tensor, paddings, mode='CONSTANT', constant_values)\ntf.pad(tensor, paddings, mode='REFLECT')\n"+\
"tf.pad(tensor, paddings, mode='SYMMETRIC')\ntf.range(start)\ntf.range(start, limit, delta)\ntf.reduce_any(input_tensor, axis)\ntf.reduce_all(input_tensor, axis)\n"+\
"tf.reduce_max(input_tensor)\ntf.reduce_max(input_tensor, axis)\ntf.reduce_mean(input_tensor)\n"+\
"tf.reduce_mean(input_tensor, axis)\ntf.reduce_min(input_tensor)\ntf.reduce_min(input_tensor, axis)\n"+\
"tf.reduce_prod(input_tensor, axis)\ntf.reduce_sum(input_tensor)\ntf.reduce_sum(input_tensor, axis)\n"+\
"tf.repeat(input, repeats)\ntf.repeat(input, repeats, axis)\ntf.reshape(tensor, shape)\n"+\
"tf.reverse(tensor, axis)\ntf.roll(input, shift, axis)\ntf.round(x)\ntf.scatter_nd(indices, updates, shape)\n"+\
"tf.searchsorted(sorted_sequence, values, side='left')\ntf.searchsorted(sorted_sequence, values, side='right')\n"+\
"tf.sequence_mask(lengths)\ntf.sequence_mask(lengths, maxlen)\ntf.shape(input)\ntf.sign(x)\n"+\
"tf.sort(values, axis)\ntf.sort(values, axis, direction='DESCENDING')\ntf.sqrt(x)\n"+\
"tf.square(x)\ntf.squeeze(input)\ntf.squeeze(input, axis)\ntf.stack(values, axis)\ntf.subtract(x, y)\n"+\
"tf.tensor_scatter_nd_update(tensor, indices, updates)\ntf.tensordot(a, b, axes)\ntf.tile(input, multiples)\n"+\
"tf.transpose(a)\ntf.transpose(a, perm)\ntf.unique_with_counts(x)\ntf.unstack(value, axis)\n"+\
"tf.where(condition)\ntf.where(condition, x, y)\ntf.zeros(shape)\ntf.zeros_like(input)"+\
"\n\nSparseTensor functions:\n-----------------------\ntf.SparseTensor(indices, values, dense_shape)\ntf.sparse.add(a, b)\n"+\
"tf.sparse.concat(axis, sp_inputs)\ntf.sparse.expand_dims(sp_input, axis)\ntf.sparse.from_dense(tensor)\ntf.sparse.maximum(sp_a, sp_b)\n"+\
"tf.sparse.minimum(sp_a, sp_b)\ntf.sparse.reduce_max(sp_input, axis, output_is_sparse)\ntf.sparse.reduce_sum(sp_input, axis, output_is_sparse)\n"+\
"tf.sparse.reset_shape(sp_input)\ntf.sparse.reshape(sp_input, shape)\ntf.sparse.retain(sp_input, to_retain)\ntf.sparse.slice(sp_input, start, size)\n"+\
"tf.sparse.split(sp_input, num_split, axis)\ntf.sparse.to_dense(sp_input)\ntf.sparse.to_dense(sp_input, default_value)\n"+\
"tf.sparse.to_indicator(sp_input, vocab_size)\ntf.sparse.transpose(sp_input)\ntf.sparse.transpose(sp_input, perm)"

In [108]:
SYSTEM_PROMPT = """You are a coding assistant. Be precise and terse.
You will be provided a list of tensorflow operators, a task description, and some input/output examples.
Your task is to generate the body of a python function that will transform the input to the output.
Only use the operators provided in the list.
"""

def make_user_message(task):
    parsed_examples: Example = task['parsed_examples']
    examples_str = "Inputs:\n"
    for inp in parsed_examples.inputs:
        examples_str += f"{inp}\n"
    examples_str += "Output:\n"
    examples_str += f"{parsed_examples.output}\n"

    formals_str = [f"in{i+1}" for i in range(len(parsed_examples.inputs))]

    return f"""[TENSORFLOW OPERATORS]
{TFOPERATORS}

[TASK DESCRIPTION]
{task['description']}

[EXAMPLES]
{examples_str}

[PROGRAM]
def transform({",".join(formals_str)}):
    """

In [113]:
def prompt(system_message: str, user_message: str, n_completions: int) -> t.List[str]:
    response = OPENAI.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message},
        ],
        n=n_completions,
        temperature=1.0,
    )
    return [choice.message.content for choice in response.choices]

In [114]:
TEST_TASK = TASK_JSONS["google_03"]

pprint(make_user_message(TEST_TASK))

prompt(SYSTEM_PROMPT, make_user_message(TEST_TASK), 5)

('[TENSORFLOW OPERATORS]\n'
 '\n'
 'TensorFlow functions to use:\n'
 '---------------------\n'
 'tf.abs(x)\n'
 'tf.add(x, y)\n'
 'tf.add_n(inputs)\n'
 'tf.argmax(input, axis)\n'
 'tf.argmin(input, axis)\n'
 'tf.argsort(values, axis, stable=True)\n'
 "tf.argsort(values, axis, direction='DESCENDING', stable=True)\n"
 'tf.boolean_mask(tensor, mask)\n'
 'tf.broadcast_to(input, shape)\n'
 'tf.cast(x, dtype)\n'
 'tf.clip_by_value(t, clip_value_min, clip_value_max)\n'
 'tf.concat(values, axis)\n'
 'tf.constant(value)\n'
 'tf.constant(value, dtype)\n'
 'tf.divide(x, y)\n'
 'tf.equal(x, y)\n'
 'tf.exp(x)\n'
 'tf.expand_dims(input, axis)\n'
 'tf.eye(num_rows)\n'
 'tf.eye(num_rows, num_columns)\n'
 'tf.eye(num_rows, dtype)\n'
 'tf.fill(dims, value)tf.gather(params, indices)\n'
 'tf.gather(params, indices, axis, batch_dims)\n'
 'tf.gather_nd(params, indices)\n'
 'tf.gather_nd(params, indices, batch_dims)\n'
 'tf.greater(x, y)\n'
 'tf.greater_equal(x, y)\n'
 'tf.math.bincount(arr)\n'
 'tf.math.ceil

['return tf.sparse.slice(sp_input= in1, start= [0, 0, 0], size= [1, -1, -1])',
 'indices = tf.boolean_mask(in1.indices, in1.indices[:, 0] == 0)\n    values = tf.gather_nd(in1.values, tf.where(in1.indices[:, 0] == 0))\n    dense_shape = tf.concat([[1], in1.dense_shape[1:]], 0)\n    return tf.SparseTensor(indices, values, dense_shape)',
 '# The task requires to slice the first dimension of a SparseTensor. We can use the `tf.sparse.slice` function for this operation.\n# Start is a 1-D tensor represents the start of the slice and for this case it will be [0, 0, 0] because we want to start from the beginning.\n# Size is a 1-D tensor represents the size of the slice. we only want to take first element along the first dimension and all elements along the remaining dimensions, so it will be [1, -1, -1].\n# tf.sparse.slice(input, start, size) function extracts a slice from a SparseTensor.\n\nreturn tf.sparse.slice(in1, start=[0, 0, 0], size=[1, -1, -1])',
 '# Function to slice the first dimensi

In [115]:
for task_name, task in TASK_JSONS.items():
    del task["completions"]

In [116]:
NUM_COMPLETIONS = 10
i = 0
for task_name, task in TASK_JSONS.items():
    if "completions" in task:
        continue 
    print(f"Prompting for task: {task['name']}, description: {task['description']}")
    completions = prompt(SYSTEM_PROMPT, make_user_message(task), NUM_COMPLETIONS)
    task["completions"] = completions
    pprint(completions)
    print()
    # i += 1
    # if i > 10:
    #     break



Prompting for task: google_01, description: Convert index tensor into pairs for SparseTensor indexing
['pairs = tf.stack([tf.math.bincount(in1), tf.range(tf.shape(in1)[0])], '
 'axis=1)\n'
 '     idx = tf.where(tf.not_equal(pairs[:, 0], 0))\n'
 '     pairs = tf.gather(pairs, idx)\n'
 '     return pairs[:, 0]',
 'return tf.stack([tf.math.floor_div(in1, 4), tf.math.floormod(in1, 4)], '
 'axis=-1)',
 'indices = tf.range(tf.shape(in1)[0])\n'
 'uniques, idx = tf.unique_with_counts(in1)\n'
 'return tf.stack([in1, tf.gather(tf.math.bincount(idx), idx)], axis=-1)',
 'return tf.stack([tf.math.bincount(in1), tf.gather(tf.range(tf.size(in1)), '
 'tf.argsort(in1))], axis=1)',
 '# We want to convert simple indices to pairs \n'
 '# First, We count the frequency of occurrence of each element in the input '
 'tensor\n'
 '# Then We create a sequence for each unique element in the input tensor up '
 'to its count\n'
 '# Finally, we gather all these sequences into our final output tensor.\n'
 '\n'
 'def 

In [117]:
TASKS_WITH_COMPLETIONS_FILE = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions.json"
for name, task in TASK_JSONS.items():
    if "parsed_examples" in task:
        del task["parsed_examples"]
TASKS_WITH_COMPLETIONS_FILE.write_text(json.dumps(DATASET, indent=4))
for name, task in TASK_JSONS.items():
    task["parsed_examples"] = Example.from_json(task["examples"])

## extract tf operators

In [118]:
def extract_tf_operators(code_snippet):
    pattern = r"tf\.[a-zA-Z_][a-zA-Z0-9_]*(?:\.[a-zA-Z_][a-zA-Z0-9_]*)*"
    return set(re.findall(pattern, code_snippet))

In [119]:
def calculate_tf_operator_coverage_and_count(target_operators, completion_operators):
    """Extend to include all completion operators and mark those used in the target program."""
    completion_operators_count = Counter(completion_operators)
    tf_operators_dict = {op: completion_operators_count[op] for op in completion_operators_count}

    # Calculate coverage based on target program operators found in completions
    covered_operators = set(target_operators).intersection(completion_operators)
    coverage_percentage = len(covered_operators) / len(target_operators) * 100 if target_operators else 0

    return {
        "tf_operators": tf_operators_dict,
        "coverage_percentage": coverage_percentage,
        "total_in_target": len(target_operators),
        "total_covered": len(covered_operators)
    }

In [120]:
for name, task in TASK_JSONS.items():
    completions = task["completions"]
    completion_tf_operators = [op for completion in completions for op in extract_tf_operators(completion)]
    target_program = task["target_program"]
    target_tf_operators = extract_tf_operators(target_program)
    tf_operator_info = calculate_tf_operator_coverage_and_count(target_tf_operators, completion_tf_operators)

    task["response"] = {
        "task_id": task.get("name", "unknown"),
        "completions": completions,
        "target-program": task["target_program"],
        "description": task["description"],
        **tf_operator_info  # Includes adjusted tf_operators dictionary
    }

In [121]:
TASKS_WITH_COMPLETIONS_FILE = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions.json"
for name, task in TASK_JSONS.items():
    if "parsed_examples" in task:
        del task["parsed_examples"]
TASKS_WITH_COMPLETIONS_FILE.write_text(json.dumps(DATASET, indent=4))
for name, task in TASK_JSONS.items():
    task["parsed_examples"] = Example.from_json(task["examples"])

In [122]:
COVERAGE_PERCENTAGES = [task["response"]["coverage_percentage"] for task in TASK_JSONS.values()]

In [123]:
# average coverage percentage
sum(COVERAGE_PERCENTAGES) / len(COVERAGE_PERCENTAGES)

64.30555555555556

In [124]:
# median coverage percentage
sorted_coverage_percentages = sorted(COVERAGE_PERCENTAGES)
median_index = len(sorted_coverage_percentages) // 2
sorted_coverage_percentages[median_index]

75.0

In [86]:
OLD_OUTPUT_FILE = CURRENT_DIRECTORY / "output_tfcoder.old.json"
OLD_OUTPUT_JSON = json.loads(OLD_OUTPUT_FILE.read_text())

OLD_COVERAGE_PERCENTAGES = [task["coverage_percentage"] for task in OLD_OUTPUT_JSON]
sum(OLD_COVERAGE_PERCENTAGES) / len(OLD_COVERAGE_PERCENTAGES)

63.95833333333334

In [125]:
# median coverage percentage
sorted_old_coverage_percentages = sorted(OLD_COVERAGE_PERCENTAGES)
median_index = len(sorted_old_coverage_percentages) // 2
sorted_old_coverage_percentages[median_index]

75.0